### Create TimeAggr for OHLC form StreamTicks

In [ ]:
TS_Filed = `code`ts`close`volume`amount
TS_Type = [SYMBOL,TIMESTAMP,DOUBLE,LONG,DOUBLE]
share streamTable(100:0,TS_Filed,TS_Type) as StreamTicks

OHLC_Filed = `ts`code`open`high`low`close`volume`amount
OHLC_Type = [TIMESTAMP, SYMBOL, DOUBLE,DOUBLE,DOUBLE,DOUBLE,LONG,DOUBLE]
share streamTable(100:0,OHLC_Filed,OHLC_Type) as OHLC

tsAggrKline = createTimeSeriesAggregator(name="Aggr_Kline", windowSize=60000, step=60000, metrics=<[first(close),max(close),min(close),last(close),sum(volume),sum(amount)]>, dummyTable=StreamTicks, outputTable=OHLC, timeColumn=`ts, keyColumn=`code)//updateTime=1,useWindowStartTime=false)
subscribeTable(tableName="StreamTicks", actionName="act_tsaggr", offset=-1, handler=append!{tsAggrKline}, msgAsTable=true)

### TickTSE TickTFE to StreamStreamTicks for OHLC

In [ ]:
def AddTsToTicks(mutable TB, msg){  
    if(msg.Date.hasNull()){
        curDate = msg.Date.max()
        if(curDate.isNull())
            curDate = today()
        msg[`Date] = msg.Date.nullFill(curDate)
    }
    data = select Code as code ,concatDateTime(Date,Time) as ts ,Close as close ,Volume as volume ,Amount as amount from msg
    TB.append!(data)
}
subscribeTable(,"StreamTickTSE", "TSE_2_Tick", -1 , AddTsToTicks{StreamTicks} , true)
subscribeTable(,"StreamTickTFE", "TFE_2_Tick", -1 , AddTsToTicks{StreamTicks} , true)

### StreamOHLC to DfsOHLC

In [ ]:
def OHLCToDfs(mutable TB, msg){  
    data = select code,ts as ts,open,high,low,close,volume,amount from msg
    TB.append!(data)
}
ohlc_tick = loadTable("dfs://ohlc",`ohlc)
subscribeTable(,"OHLC", "act_ohlc2dfs", -1, OHLCToDfs{ohlc_tick} , true)